## Break repeating-key XOR
### It is officially on, now.

This challenge isn't conceptually hard, but it involves actual error-prone coding. The other challenges in this set are there to bring you up to speed. This one is there to qualify you. If you can do this one, you're probably just fine up to Set 6.


[There's a file here](https://www.cryptopals.com/static/challenge-data/6.txt). It's been base64'd after being encrypted with repeating-key XOR.

Decrypt it. 

 Here's how:

1. Let KEYSIZE be the guessed length of the key; try values from 2 to (say) 40.
2. Write a function to compute the edit distance/Hamming distance between two strings. _The Hamming distance is just the number of differing bits_. The distance between:

    `this is a test`

    and

    `wokka wokka!!!`

    is *37*. _Make sure your code agrees before you proceed_.

2. For each KEYSIZE, take the first KEYSIZE worth of bytes, and the second KEYSIZE worth of bytes, and find the edit distance between them. Normalize this result by dividing by KEYSIZE.
2. The KEYSIZE with the smallest normalized edit distance is probably the key. You could proceed perhaps with the smallest 2-3 KEYSIZE values. Or take 4 KEYSIZE blocks instead of 2 and average the distances.
2. Now that you probably know the KEYSIZE: break the ciphertext into blocks of KEYSIZE length.
2. Now transpose the blocks: make a block that is the first byte of every block, and a block that is the second byte of every block, and so on.
2. Solve each block as if it was single-character XOR. You already have code to do this.
2. For each block, the single-byte XOR key that produces the best looking histogram is the repeating-key XOR key byte for that block. Put them together and you have the key.

This code is going to turn out to be surprisingly useful later on. Breaking repeating-key XOR ("Vigenere") statistically is obviously an academic exercise, a "Crypto 101" thing. But more people "know how" to break it than can _actually break it_, and a similar technique breaks something much more important. 

### No, that's not a mistake.

We get more tech support questions for this challenge than any of the other ones. We promise, there aren't any blatant errors in this text. In particular: the "wokka wokka!!!" edit distance really is 37.


In [6]:
test = b"this is a test"
wokka = b"wokka wokka!!!"

### DIY Hamming distance

In [2]:
def hamming_distance(s1, s2):
    count = 0
    for ch1, ch2 in zip(s1, s2):
        if ch1 != ch2:
            count += 1
    return count

In [3]:
count = 0
for b1, b2 in zip(test, wokka):
    count += hamming_distance(format(b1, '08b'), format(b2, '08b'))
count

37

In [4]:
x = 29
bin(x)

'0b11101'

In [5]:
format(x, '08b')

'00011101'

### SciPy Hamming distance

In [1]:
from scipy.spatial import distance

In [2]:
distance.hamming([1, 0, 0], [0, 1, 0])

0.6666666666666666

In [3]:
distance.hamming([1, 0, 0], [1, 1, 0])

0.3333333333333333

In [4]:
def to_1d_array(byte_str):
    tmp = ''
    for b in byte_str:
        tmp += format(b, '08b')
    return [int(ch) for ch in tmp]

In [7]:
d = distance.hamming(to_1d_array(test), to_1d_array(wokka))
d

0.33035714285714285

In [8]:
len(test) * 8 * d

37.0